In [272]:
import numpy as np
import pandas as pd

df = pd.DataFrame()
df = pd.read_csv('C:/Users/Danny/Documents/fall17/Data Science/train.csv')
df2 = pd.DataFrame()
df2 = pd.read_csv('C:/Users/Danny/Documents/fall17/Data Science/test.csv')
#only use first 10,000 so it works faster
df = df.head(10000)
df.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [273]:
import datetime as datetime
#changes format to dt(datetime) rather than string
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
df2['pickup_datetime'] = pd.to_datetime(df2['pickup_datetime'])
#change datetime column to new columns dayofweek, hour, and minute of day
#new columns have numerical values rather than a string 'datetime',
#which Linear Regression can't process
df['day_of_week'] = df['pickup_datetime'].dt.weekday_name
df['hour'] = df['pickup_datetime'].dt.hour
df2['day_of_week'] = df2['pickup_datetime'].dt.weekday_name
df2['hour'] = df2['pickup_datetime'].dt.hour
#minute of day is 60*hour of day + minutes, works well with regression
df['minute_of_day'] = (df['pickup_datetime'].dt.hour*60)+df['pickup_datetime'].dt.minute
df2['minute_of_day'] = (df2['pickup_datetime'].dt.hour*60)+df2['pickup_datetime'].dt.minute
#month as a value of 0-11 
df['month'] = df['pickup_datetime'].dt.month
df2['month'] = df2['pickup_datetime'].dt.month
import calendar
#Assign names to month so OneHotEncoding doesn't overlap integer values
df['month'] = df['month'].apply(lambda x: calendar.month_abbr[x])
df2['month'] = df2['month'].apply(lambda x: calendar.month_abbr[x])
#day of year as numerical (0-364) to account for New Years and other busy days
df['day_of_year'] = df['pickup_datetime'].dt.dayofyear
df2['day_of_year'] = df2['pickup_datetime'].dt.dayofyear
#calculate haversine distance between two (long,lat) pairs
from sklearn.neighbors import DistanceMetric
dist = DistanceMetric.get_metric('haversine')
l = []
for i in range(0,len(df)):
    X = [[df['pickup_latitude'][i],df['pickup_longitude'][i]],
         [df['dropoff_latitude'][i],df['dropoff_longitude'][i]]]
    l.append(dist.pairwise(X)[0][1])
r = []
for i in range(0,len(df2)):
    X = [[df2['pickup_latitude'][i],df2['pickup_longitude'][i]],
         [df2['dropoff_latitude'][i],df2['dropoff_longitude'][i]]]
    r.append(dist.pairwise(X)[0][1])
#create new dataframe to add calculated array to
df3 = pd.DataFrame()
df3['distance'] = l
df = df.join(df3['distance'])
df4 = pd.DataFrame()
df4['distance'] = l
df2 = df2.join(df4['distance'])
#select features that don't need oneHotEncoding
#X = df[['pickup_latitude','pickup_longitude','dropoff_latitude','dropoff_longitude','passenger_count', 'minute_of_day','day_of_year','distance']]
X = df[[]]
X2 = df[[]]
#normalize
normalized_X=(X - X.mean())/X.std()
normalized_X2=(X2 - X2.mean())/X2.std()
#normalized_X= (X)/X.max()
#day of week and month get one hot encoded because they do not hold comparablevalues in numerical order
oh_dw =pd.get_dummies(df['day_of_week'])
oh_m =pd.get_dummies(df['month'])
oh_dw2 =pd.get_dummies(df2['day_of_week'])
oh_m2 =pd.get_dummies(df2['month'])
#add onehotencoded columns to normalized feature set
normalized_X = normalized_X.join(oh_dw)
normalized_X2 = normalized_X2.join(oh_dw2)
#normalized_X = normalized_X.join(oh_m)
normalized_X.head()
normalized_X2[np.isnan(normalized_X2)] = np.median(normalized_X2[~np.isnan(normalized_X2)])

In [264]:
from sklearn.model_selection import train_test_split
y = df['trip_duration']
#X_train, X_test, y_train, y_test = train_test_split(normalized_X, y, test_size=0.4, random_state=2)


In [265]:
#Linear Regression
from sklearn.linear_model import LinearRegression
import math
my_linreg = LinearRegression()
my_linreg.fit(X_train, y_train)
y_prediction = my_linreg.predict(X_test)
#mse = metrics.mean_squared_log_error(y_prediction, y_test)
total = 0
x=0
for i, row in y_test.iteritems():
    if(y_prediction[x]<=0): y_prediction[x] = 0
    total = total + math.pow((math.log(y_prediction[x]+1)-math.log(i+1)),2)
    x = x+1
    
print(my_linreg.intercept_)
co = my_linreg.coef_
for x in range(0, len(co)):
    print(x, '%f' % co[x])

math.sqrt(total/len(y_test))


932.109600787
0 23.646472
1 -13.920159
2 -43.676069
3 -37.822831
4 533.949267
5 -0.937552
6 -89.651124
7 39.371550
8 -118.363968
9 59.170342
10 78.488810
11 31.921942


3.917626360006948

In [228]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2)
X_ = poly.fit_transform(normalized_X)
my_polyreg = LinearRegression()


In [52]:
from sklearn.model_selection import cross_val_score
mse_list = cross_val_score(my_linreg, normalized_X, y, cv=10, scoring='neg_mean_squared_error')
mse_list_positive = -mse_list
rmse_list = np.sqrt(mse_list_positive)
mse_list = cross_val_score(my_polyreg, X_, y, cv=10, scoring='neg_mean_squared_error')
mse_list_positive3 = -mse_list
rmse_list3 = np.sqrt(mse_list_positive3)
print("Linear: ",rmse_list.mean(),"   poly: ",  rmse_list3.mean())

Linear:  1987.48156554    poly:  2403.29384956
